<h1>CS152 Assignment 2: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [13]:

class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle

    Attributes:
        puzzle (list[list[int]]): 
            Represents the puzzle state as a (2D) list of lists of integers. 0 represents the blank tile. E.g. [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
        parent (PuzzleNode): 
            The parent node. None if this is the root node
        action (tuple): 
            The action that was taken to get to this node. None if this is the root node. Useful for printing the solution path.
            Can be one of the following (0, 1), (0, -1), (1, 0), (-1, 0) where the first value is the row offset and the second value is the column offset of the parent node.
            Tuples represent moving the blank tile to the right, left, down, or up respectively. 
        path_cost (int): 
            The path cost to get to this node. 0 if this is the root node. Necessary for calculating the total cost of a node in A* search.
        size (int): 
            The size of the puzzle (i.e. the value of n in an n^2-1 puzzle)
        blank_coord (tuple[int, int]): 
            The coordinates of the blank tile in the puzzle. Format is (row, col). 

    Methods:
        get_blank_coord: 
            Return the coordinates (row, col) of the blank tile in the puzzle
        __str__: 
            Return the string representation of the puzzle
        __eq__: 
            Return True if the puzzle states of self and other are equal, False otherwise
        __lt__: 
            Return True if the puzzle states of self is less than the puzzle state of other, False otherwise
        __hash__:
            Return the hash of the puzzle state
    """

    def __init__(self, puzzle, parent, action, path_cost):
        """
        Initialize a puzzle node with the given puzzle, parent, action, path_cost, size, and blank_coord.
        """
        self.puzzle = puzzle
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.size = len(puzzle)
        self.blank_coord = self.get_blank_coord()

    def get_blank_coord(self):
        """
        Return the coordinates of the blank tile in the puzzle.
        """
        for i in range(self.size):
            for j in range(self.size):
                if self.puzzle[i][j] == 0:
                    return (i, j)
        return None

    def __str__(self):
        """
        Return the string representation of the puzzle.
        """
        return str(self.puzzle)

    def __eq__(self, other):
        """
        Return True if the puzzle states of self and other are equal, False otherwise.
        """
        return self.puzzle == other.puzzle

    def __lt__(self, other):
        """
        Return True if the puzzle states of self is less than the puzzle state of other, False otherwise. Necessary for using the heapq module.
        """
        return self.puzzle < other.puzzle

    def __hash__(self):
        """
        Return the hash of the puzzle state. 
        """
        return hash(str(self.puzzle))


<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [14]:
# Misplaced tiles heuristic
def h1(state):
    """
    This function returns the number of misplaced tiles, given the board state.
    Compares each tile in the current state with the tile at the same location in the goal state and increments the number of misplaced tiles if they are not equal.

    Input:
        -state: the board state as a list of lists
        -heuristics: a dictionary to store the heuristic values previously evaluated for a given state
    Output:
        -h: the number of misplaced tiles
    """
    h = 0
    # iterate through each tile in the state
    for i in range(len(state)):
        for j in range(len(state)):
            # if the value of the tile is not 0 and not equal to the value of the goal state which is the 2d matrix index (i*len(state) + j)
            if state[i][j] != 0 and state[i][j] != (i*len(state) + j):
                h += 1  # increment the number of misplaced tiles
    return h


# Manhattan distance heuristic
def h2(state):
    """
    This function returns the sum of the Manhattan distances from the solved state, given the board state.
    Calculates the sum of the distances of each tile from its goal position using the Manhattan distance formula: |x1-x2| + |y1-y2|


    Input:
        -state: the board state as a list of lists
    Output:
        -h: total Manhattan distance from the solved configuration
    """
    h = 0

    for i in range(len(state)):
        for j in range(len(state)):
            # if the value of the tile is not 0 and not in the correct position which is the 2d matrix index (i*len(state) + j)
            if state[i][j] != 0 and state[i][j] != (i*len(state) + j):
                # calculate the distance between the current position and the correct position
                # equivalent to |x1-x2| + |y1-y2|
                h += abs(i - state[i][j]//len(state)) + \
                    abs(j - state[i][j] % len(state))

    return h

# Pattern database function for n=3


def pattern_database(n):
    """
    This function returns a pattern database as a dictionary that maps a pattern (a tuple of tile positions) to its minimum distance to the goal state.
    It uses BFS to find all patterns for the 8 puzzle. Only viable for n=3 due bad time complexity. For n=4, another 

    Inputs:
        - n: the size of the puzzle
    Outputs:
        - pattern_database: a pattern database as a dictionary that maps a pattern (a tuple of tile positions) to its minimum distance to the goal state. E.g. {(0, 1, 2, 3): 0, (0, 1, 3, 2): 1, (0, 2, 1, 3): 2, ...}
    """
    from copy import deepcopy

    n = 3

    pattern_database = {}  # Initialize the pattern database

    # Initialize the goal state
    goal_state = [[i for i in range(j*n, (j+1)*n)] for j in range(n)]

    queue = []  # Initialize the queue
    visited = set()  # Initialize the visited set

    # Initialize the pattern database with the goal state
    pattern_database[tuple([tuple(row) for row in goal_state])] = 0
    queue.append(goal_state)  # Add the goal state to the queue
    # Add the goal state to the visited set
    visited.add(tuple([tuple(row) for row in goal_state]))

    moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Initialize the action space

    while queue:  # While the queue is not empty

        # Pop the first state from the queue
        state = queue.pop(0)

        # Get the pattern of the state
        pattern = tuple([tuple(row) for row in state])

        # Get the distance of the state from the goal state
        distance = pattern_database[pattern]

        # find the coordinates of the blank space
        for i in range(n):
            for j in range(n):
                if state[i][j] == 0:
                    blank_row, blank_col = i, j
                    break

        # For each action in the action space
        for move in moves:

            # new row, column coordinates of the blank space
            new_row, new_col = blank_row + move[0], blank_col + move[1]

            # if the new coordinates are within the puzzle
            if (new_row >= 0 and new_row < n) and (new_col >= 0 and new_col < n):

                # copy the state so that the original state is not modified
                new_state = deepcopy(state)

                # swap the blank space with the tile in the new coordinates
                new_state[blank_row][blank_col], new_state[new_row][new_col] = new_state[new_row][new_col], new_state[blank_row][blank_col]
                # get the pattern of the new state. tuple since it is hashable
                new_pattern = tuple([tuple(row) for row in new_state])

                if new_pattern not in visited:  # if the new pattern has not been visited
                    # add the new pattern to the pattern database with the distance of the new state from the goal state
                    pattern_database[new_pattern] = distance + 1
                    queue.append(new_state)
                    visited.add(new_pattern)

    # save the pattern database to a file
    with open('pattern_database_3.txt', 'w') as f:
        f.write(str(pattern_database))
        print('Pattern database saved to file.')

    # Return the pattern database
    return pattern_database


global pattern_database_3
# read the pattern database from a file if it exists
try:
    with open('pattern_database_3.txt', 'r') as f:
        pattern_database_3 = eval(f.read())
        print('Pattern database loaded from file.')
except:
    # otherwise generate the pattern database
    pattern_database_3 = pattern_database(3)
    print('Pattern database generated using the pattern generation function.')


def static_additive_disjoint_pattern_database():
    """
    This function returns a static additive disjoint pattern database as a dictionary that maps a 
    pattern (a tuple of tile positions) to its minimum distance to the goal state for the 15 puzzle. 
    It uses 6-6-3 partitioning to split the possibilities into 3 different subproblems. It uses BFS 
    to find all patterns for these subproblems. 

    It takes a very long time to load everything for the first time but you only have to run it once 
    after which is saved to a file and loaded from the file even after restarting the kernel. It signi-
    ficantly reduces the time complexity. I have obtained around a 3500x speedup and solved many that
    were unsolvable before.

    Inspired by: 
    - Additive Pattern Database Heuristic with Static 6-6-3 Partitioning A. Felner, R. Korf, and S. Hanan,
      "Additive Pattern Database Heuristics," Journal of AI Research, Vol. 22, pp. 279-318, 2004.
    - https://algorithmsinsight.wordpress.com/graph-theory-2/a-star-in-general/implementing-a-star-to-solve-n-puzzle/
    - https://algorithmsinsight.wordpress.com/graph-theory-2/implementing-bfs-for-pattern-database/
    - https://michael.kim/blog/puzzle
    - https://web.archive.org/web/20141224035932/http://juropollo.xe0.ru:80/stp_wd_translation_en.htm

    Outputs:
        - static_additive_disjoint_pattern_database (a dictionary [key: (tuple) representing a pattern, value: (int) minimum distance to the goal state]) 
            Returns one for each of the subsets: 

    """
    # read file if it exists
    try:
        with open('static_additive_disjoint_pattern_database.txt', 'r') as f:
            static_additive_disjoint_pattern_database = eval(f.read())
            print('Static additive disjoint pattern database loaded from file.')
    except:
        print('Static additive disjoint pattern database not found. Generating static additive disjoint pattern database...')

        from copy import deepcopy

        n = 4

        """ creating dictionaries for each subset that store:
            - the values of the tiles that are looked at in the subset. e.g. [1, 2, 4, 5, 8, 9] for subset 1
            - the goal state (2d list) of the subset (how the goal state of the values above should look like). -1 is a placeholder for ignored tiles
            - the indexes of the values in the goal state. Tuple of tuples (row, col) for each of the values observed. 
                e.g. ((0, 1), (0, 2), (1, 0), (1, 1), (2, 0), (2, 1)) for subset 1
            - the distances of the goal state from itself. 
                Dictionary that maps a pattern (tuple of the observed tile indexes) to its minimum distance to the goal state.

        """

        subset_1_goalstate = {'values': [1, 2, 4, 5, 8, 9],
                              'goalstate': [[0, 1, 2, -1],
                                            [4, 5, -1, -1],
                                            [8, 9, -1, -1],
                                            [-1, -1, -1, -1]],
                              'goalstate_indexes': ((0, 1), (0, 2), (1, 0), (1, 1), (2, 0), (2, 1)),
                              'distances': {((0, 1), (0, 2), (1, 0), (1, 1), (2, 0), (2, 1)): 0}
                              }

        subset_2_goalstate = {'values': [3, 6, 7, 10, 11, 15],
                              'goalstate': [[0, -1, -1, 3],
                                            [-1, -1, 6, 7],
                                            [-1, -1, 10, 11],
                                            [-1, -1, -1, 15]],
                              'goalstate_indexes': ((0, 3), (1, 2), (1, 3), (3, 2), (3, 3), (4, 3)),
                              'distances': {((0, 3), (1, 2), (1, 3), (3, 2), (3, 3), (4, 3)): 0}
                              }

        subset_3_goalstate = {'values': [12, 13, 14],
                              'goalstate': [[0, -1, -1, -1],
                                            [-1, -1, -1, -1],
                                            [-1, -1, -1, -1],
                                            [12, 13, 14, -1]],
                              'goalstate_indexes': ((3, 0), (3, 1), (3, 2)),
                              'distances': {((3, 0), (3, 1), (3, 2)): 0}
                              }

        def get_subset_positions(state, values_to_find):
            """Traverses through the state and returns the positions of the values in the values_to_find list

            Inputs:
                - state (list of lists): the state of the puzzle
                - values_to_find (list): the values to find in the state. E.g. [1, 2, 4, 5, 13]
            Outputs:
                - positions (tuple): the positions of the values in the state. E.g. ((0, 1), (0, 2), (1, 0), (1, 1), (3, 2))
            """

            # initialize the dictionary
            values_poistion_dict = {value: (0, 0) for value in values_to_find}

            # traverse through the state and add the positions of the values to the dictionary
            for i in range(n):
                for j in range(n):
                    if state[i][j] in values_to_find:
                        values_poistion_dict[state[i][j]] = (i, j)

            # return the positions of the values in the state from the dictionary as a tuple
            return tuple(values_poistion_dict.values())
        
        # initialize the action space
        moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]

        for subset in [subset_1_goalstate, subset_2_goalstate, subset_2_goalstate]: # for each of the subsets
            # BFS process
            queue = [] # initialize the queue
            visited = set() # initialize the visited set

            # Add the goal state to the queue
            queue.append(subset['goalstate'])
            # Add the goal state to the visited set
            visited.add(tuple([tuple(row) for row in subset['goalstate']]))

            # While the queue is not empty
            while queue:
                # Pop the first state from the queue
                state = queue.pop(0)
                indexes = get_subset_positions(state, subset['values'])
                distance = subset['distances'][indexes]

                # get the position of the blank space
                for i in range(n):
                    for j in range(n):
                        if state[i][j] == 0:
                            blank_row, blank_col = i, j
                            break

                # For each action in the action space
                for move in moves:
                    new_row, new_col = blank_row + move[0], blank_col + move[1] # get the new position of the blank space
                    if (new_row >= 0 and new_row < n) and (new_col >= 0 and new_col < n): # check if the new position is valid
                        new_state = deepcopy(state) # create a copy of the state
                        # switch the blank space with the tile in the new position
                        new_state[blank_row][blank_col], new_state[new_row][new_col] = new_state[new_row][new_col], new_state[blank_row][blank_col]
                        indexes = get_subset_positions(
                            new_state, subset['values']) # get the positions of the tiles in the subset

                        # check if not in visited
                        if tuple([tuple(row) for row in new_state]) not in visited:
                            # check if blank switched with a tile that has a value of -1
                            if new_state[new_row][new_col] == -1: # if yes, then the distance is 1 more than the parent
                                if indexes in subset['distances']: # check if the distance is already in the dictionary
                                    subset['distances'][indexes] = max(
                                        distance, subset['distances'][indexes]) # update the distance if it is less than the current distance
                                else: # if the distance is not in the dictionary, then add it
                                    subset['distances'][indexes] = distance # add the distance to the dictionary
                            else: # if the blank space switched with a tile that has a value other than -1, then the distance is 1 more than the parent
                                if indexes in subset['distances']: # check if the distance is already in the dictionary
                                    subset['distances'][indexes] = max(
                                        distance+1, subset['distances'][indexes]) # update the distance if it is less than the current distance
                                else:
                                    subset['distances'][indexes] = distance + 1 # add the distance to the dictionary

                            queue.append(new_state) # add the new state to the queue
                            visited.add(tuple([tuple(row) 
                                        for row in new_state])) # add the new state to the visited set
            print("Subset finished. ")

        # save the pattern database to a pkl file so you only have to run it once
        import pickle
        with open('pattern_database.pkl', 'wb') as f:
            pickle.dump(
                [subset_1_goalstate, subset_2_goalstate, subset_3_goalstate], f)
            print('Static additive disjoint pattern database saved to file.')

        return subset_1_goalstate['distances'], subset_2_goalstate['distances'], subset_3_goalstate['distances']


def h3(state):
    """
    This function returns a heuristic that dominates the Manhattan distance, given the board state and a pattern database.

    Inputs:
        - state: the board state as a list of lists
        - pattern_database: a pattern database as a dictionary that maps a pattern (a tuple of tile positions) to its minimum distance to the goal state
    Outputs:
        - h: the heuristic distance of the state from its solved configuration
    """
    pattern_database = pattern_database_3
    puzzle_size = len(state)

    if puzzle_size == 3:
        h = pattern_database[tuple([tuple(row) for row in state])]
    else:
        h = h2(state)

    return h


# If you implement more than 3 heuristics, then add any extra heuristic functions onto the end of this list.
heuristics = [h1, h2, h3]

Pattern database loaded from file.


<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [17]:
# Import any packages or define any helper functions you need here
import heapq  # for priority queue
from copy import deepcopy


def is_correct_format(state):
    """
    This function checks if the given state is in the correct format: nxn square with numbers from 0 to n**2-1 occurring exactly once.

    Inputs:
        -state: the board state as a list of lists
    Outputs:
        -True if the state is in the correct format, False otherwise.
    """
    puzzle_size = len(state)  # the number of rows in the puzzle
    # check if the puzzle is a square
    # if any row is not the same length as the puzzle size
    if not all(len(row) == puzzle_size for row in state):
        return False
    # check if the puzzle contains all the numbers from 0 to n**2-1
    goal_state = [i*puzzle_size +
                  j for i in range(puzzle_size) for j in range(puzzle_size)]
    # sorting the state should give the ideal state
    if sorted([num for row in state for num in row]) != goal_state:
        return False
    return True


def children(node):
    """
    This function takes a node and generates all valid children to that node.
    Inputs:
        -node: a PuzzleNode object
    Outputs:
        -children (list[PuzzleNode]): a list of PuzzleNode objects that are valid children to the given node
    """
    children = []  # list of PuzzleNode objects that are valid children to the given node
    puzzle_size = node.size  # size of the puzzle (e.g. 5 for a 5x5 puzzle)
    # coordinates of the blank space in the puzzle
    blank_row, blank_col = node.blank_coord
    # each move is a tuple (row, col). possible moves: right, left, down, up.
    moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    for move in moves:  # for each possible move
        # new row, column coordinates of the blank space
        new_row, new_col = blank_row + move[0], blank_col + move[1]
        # if the new coordinates are within the puzzle boundaries.
        # e.g. if the blank space is in the top left corner (0,0), then the new coordinates cannot be (-1, -1) or (-1, 0) or (0, -1)
        if (new_row >= 0 and new_row < puzzle_size) and (new_col >= 0 and new_col < puzzle_size):
            # create a copy of the puzzle to avoid changing the original puzzle
            new_puzzle = deepcopy(node.puzzle)
            # swap the blank space with the tile in the new coordinates
            new_puzzle[blank_row][blank_col], new_puzzle[new_row][new_col] = new_puzzle[new_row][new_col], new_puzzle[blank_row][blank_col]
            # create a new PuzzleNode object form this and add it to the list of children
            children.append(PuzzleNode(
                puzzle=new_puzzle, parent=node, action=move, path_cost=node.path_cost+1))

    return children


def is_goal(node):
    """
    This function checks if the given node is the goal state.

    Inputs:
        -node: a PuzzleNode object
    Outputs:
        -bool: True if the node is the goal state, False otherwise.
    """
    puzzle_size = node.size  # size of the puzzle (e.g. 5 for a 5x5 puzzle)
    goal_state = [
        [i*puzzle_size+j for j in range(puzzle_size)] for i in range(puzzle_size)]  # [[0, 1, 2], [3, 4, 5], [6, 7, 8]] if 3x3 puzzle
    # check if the puzzle of the node is the same as the goal state. Return True if it is, False otherwise.
    return node.puzzle == goal_state


def is_solvable(node):
    """
    This function checks if the given node's puzzle is solvable using the inversion count theory. 
    Idea from: https://www.youtube.com/watch?v=bhmCmbj9VAg retrieved on 1/4/2023

    The function uses the inversion count theory to determine the solvability of the puzzle. 
    Firstly, an inversion is defined as a pair of tiles (a,b) where a appears before b in the flattened list of tiles but a > b. 
    E.g., in the lsist [3, 1, 2, 4, 5, 6, 7] there are two inversions: (3,1), (3,2).
    Now, if the number of inversions in the puzzle state is even, the puzzle is solvable because for each inversion, 
    we need to make an odd number of moves to get the tiles in the correct order. However, if there are an even number of inversions,
    we can always make an even number of moves to get all the tiles in the correct order. If the number of inversions is odd, 
    the puzzle is not solvable.  This is because we need to make an odd number of moves for each inversion,
    but we only have an even number of moves available (moving the tiles around the blank tile). Therefore, it is impossible to get all the 
    tiles in the correct order.     Now, if the puzzle size is odd, then the blank tile is always in an even position. 
    Thus,the function checks if the number of inversions is even and if it is, then the puzzle is solvable, otherwise, it is not solvable.
    If the puzzle size is even, the solvability of the puzzle depends on the position of the blank tile. 
    Specifically, the solvability of the puzzle depends on the parity of the sum of the distance of the blank tile from its 
    correct position and the number of inversions. 
    The distance of the blank tile from its correct position is calculated as the number of rows 
    from the bottom row to the blank tile position. If this distance is even, then the number of
    inversions must be odd for the puzzle to be solvable. If the distance is odd, then the number of 
    inversions must be even for the puzzle to be solvable.

    Inputs:
        -node: a PuzzleNode object
    Outputs:
        -True if the state is solvable, False otherwise.
    """
    inversions = 0
    puzzle_size = node.size  # size of the puzzle (e.g. 5 for a 5x5 puzzle)
    # Find the coordinates of the blank tile.
    blank_row, _ = node.blank_coord

    # Flatten the puzzle state.
    # converts the matrix to 1D list e.g. [[0,1], [2,3]] -> [1,2,3]
    flat_state = [num for row in node.puzzle for num in row if num != 0]
    for i in range(len(flat_state)):  # for each number in the list
        for j in range(i+1, len(flat_state)):  # for each number after the current number
            # if the current number is bigger than the next number, we have an inversion
            if flat_state[i] > flat_state[j]:
                inversions += 1

    if puzzle_size % 2 == 1:  # if the puzzle size is odd
        return inversions % 2 == 0
    else:  # if the puzzle size is even
        # Calculate the distance of the blank tile from the bottom.
        dist_from_bottom = puzzle_size - blank_row - 1
        if dist_from_bottom % 2 == 0:  # if the distance is even
            # then the number of inversions should be odd. if it is even, the puzzle is not solvable
            return inversions % 2 == 1
        else:  # if the distance is odd
            # then the number of inversions should be even. if it is odd, the puzzle is not solvable
            return inversions % 2 == 0


# Main solvePuzzle function.
def solvePuzzle(state, heuristic):
    """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).

    A PuzzleNode object that represents the starting state and has zero route cost, no action, and no parent is
    first created by the algorithm. Then, if the beginning state is not solvable (i.e., cannot be reached from
      the initial state to the desired state), an error code is returned. It creates an empty explored set and 
      an empty frontier (a priority queue) (a set to memoize the visited nodes).

    As soon as the frontier is empty, the algorithm runs once again (i.e., all possible states have been explored).
      The node with the lowest predicted total cost (heuristic plus path cost) is removed from the frontier after 
      each loop iteration. The algorithm creates the best path if this node is the goal state by going back through
        the parent nodes and the steps taken to get there.

    The technique widens the examined set by creating all feasible children nodes if the node is not the desired 
    state (i.e., all possible moves from the current state). Each child node's heuristic value is calculated (either by utilising
      the cached value or by calculating it), and if it hasn't been visited previously, it is added to the frontier.

    If the node is not in the desired state, the approach expands the examined set by producing all feasible children
      nodes (i.e., all possible moves from the current state). If a child node hasn't been visited before, its heuristic
        value is calculated for each one (either by using the cached value or by calculating it), and it is added to the frontier.
    
    Inputs:
        -state: The initial state of the puzzle as a list of lists (e.g. [[1,2,3],[4,5,6],[7,8,0]])
        -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
    Outputs:
        -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
        -exp: The number of nodes expanded to reach the solution
        -max_frontier: The maximum size of the frontier over the whole search
        -opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of lists
                    that represents the state of the board at the ith step of the solution.
        -err: An error code.  If state is not of the appropriate size and dimension, return -1.  For the extention task,
          if the state is not solvable, then return -2
    """

    h_cache = {}  # cache for the heuristic values

    # Check if the input is in the correct format
    if not is_correct_format(state):
        return 0, 0, 0, None, -1  # Error code for incorrect input

    # Create the initial node with zero path cost and null action and parent
    start_node = PuzzleNode(state, parent=None, action=None, path_cost=0)

    # Check if the initial state is solvable
    if not is_solvable(start_node):
        return 0, 0, 0, None, -2  # Error code for unsolvable input

    # Initialize the frontier and the explored set
    frontier = []
    # Priority is the estimated total cost (heuristic + path cost)
    heapq.heappush(frontier, (heuristic(state) + 0,
                   start_node))  # (priority, node)
    explored = set()  # memoize the explored nodes

    # Initialize variables to keep track of the maximum frontier size and optimal path
    max_frontier = 1
    opt_path = []  # containts the optimal path as a list of list of lists. e.g. [[[0,1], [2,3]], [[1,0], [2,3]]]
    # dictionary to map the action to the move made
    moves_dict = {(0, 1): 'R', (0, -1): 'L', (1, 0): 'D', (-1, 0): 'U'}
    moves_made = []  # list to store the moves made e.g. ['R', 'D', 'L', 'U']

    while len(frontier) > 0:  # while the frontier is not empty (i.e. there are still nodes to explore)
        # Pop the node with the lowest estimated total cost
        _, node = heapq.heappop(frontier)  # (priority, node)

        # Check if the node is the goal state
        if is_goal(node):
            # Generate the optimal path by tracing back through the parent nodes
            # while the current node is not the initial state (only the initial state / root node has no parent)
            while node.parent is not None:
                # append the current state to the optimal path
                opt_path.append(node.puzzle)
                # append the move made to the list of moves made
                moves_made.append(moves_dict[node.action])
                node = node.parent  # move to the parent node
            opt_path.append(node.puzzle)  # append the initial state
            moves_made.reverse()  # reverse the moves to get the correct order
            # print('Moves made: ', moves_made) # uncomment to print the moves made e.g. ['R', 'D', 'L', 'U']
            opt_path.reverse()  # reverse the path to get the correct order

            # Return the number of steps, number of nodes expanded, maximum frontier size, optimal path and error
            return len(opt_path) - 1, len(explored), max_frontier, opt_path, 0

        # Add the current node to the explored set
        explored.add(node)

        # Expand the current node by generating all possible children nodes
        for child_node in children(node):
            if child_node not in explored:
                # get the heuristic value from the cache if it exists, otherwise calculate it
                if child_node in h_cache:
                    h_val = h_cache[child_node]
                else:
                    h_val = heuristic(child_node.puzzle)
                    h_cache[child_node] = h_val

                # Add the child node to the frontier if it has not been visited before
                # Priority is the estimated total cost (heuristic + path cost)
                heapq.heappush(
                    frontier, (h_val + child_node.path_cost, child_node))

        # Update the maximum frontier size if the current frontier size is greater than the maximum frontier size
        max_frontier = max(max_frontier, len(frontier))

    return 0, 0, 0, None, -1  # Error code for failure to find a solution


<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [18]:
# Test for state not correctly defined

incorrect_state = [[0, 1, 2], [2, 3, 4], [5, 6, 7]]
_, _, _, _, err = solvePuzzle(incorrect_state, lambda state: 0)
assert (err == -1)


In [19]:
# Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2, 3, 7], [1, 8, 0], [6, 5, 4]], [
                                   [7, 0, 8], [4, 6, 1], [5, 3, 2]], [[5, 7, 6], [2, 4, 3], [8, 1, 0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7, 8, 7]
h_man_vals = [15, 17, 18]

for i in range(0, 3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert (h_mt == h_mt_vals[i])
    assert (h_man == h_man_vals[i])


In [20]:
# A* Tests for 3 x 3 boards
# This test runs A* with both heuristics and ensures that the same optimal number of steps are found
# with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]],
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [
                     [2, 3, 7], [1, 0, 4], [6, 8, 5]],
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [
                     [0, 2, 7], [1, 3, 4], [6, 8, 5]],
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [
                     [1, 2, 7], [3, 0, 4], [6, 8, 5]],
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [
                     [1, 2, 0], [3, 4, 7], [6, 8, 5]],
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [
                     [1, 4, 2], [3, 0, 7], [6, 8, 5]],
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [
                     [1, 4, 2], [3, 7, 5], [6, 8, 0]],
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [
                     [1, 4, 2], [3, 0, 5], [6, 7, 8]],
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0, 3):
    steps_mt, expansions_mt, a, opt_path_mt, err1 = solvePuzzle(
        working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, b, opt_path_man, err2 = solvePuzzle(
        working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert (steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert (expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert (opt_path_mt == opt_path_soln)


In [21]:
# A* Test for 4 x 4 board
# This test runs A* with both heuristics and ensures that the same optimal number of steps are found
# with each heuristic.

working_initial_state_15_puzzle = [[1, 2, 6, 3], [
    0, 9, 5, 7], [4, 13, 10, 11], [8, 12, 14, 15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(
    working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(
    working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert (steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert (expansions_mt >= expansions_man)


<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [22]:
# Puzzle solvability test

"""
Explained in the function is_solvable()
"""

unsolvable_initial_state = [[7, 5, 6], [2, 4, 3], [8, 1, 0]]
_, _, _, _, err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert (err == -2)


Implemented Additive Disjoint Pattern Database Heuristic, which dominates Manhattan distance since it gives the minimal distance to the goal state. It is calculated by starting from the goal state and then using bfs to see all the patterns always saving the minimum distance taken. Although it takes long to generate it initially, it significantly reduces the time complexity of the heuristic function, making it the fastest for n>=4.  Although there exist many partitions for the disjoint pattern database, I have used the one which research suggests is the perfect balance between memory and speed: 6-3-3.

Research suggests that Dynamically partitioned Additive Pattern Database might be an improvement so that can be implemented in the future. 



In [23]:
# Extra heuristic function test.
# This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.



dom = 0
for i in range(0, 3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(
        working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(
        working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert (steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
    assert (dom > 0)


In [24]:
# Memoization test - will be carried out after submission

""" 
Memoization implemented in the function solvePuzzle() by using a dictionary to storing the already calculated values of the heuristic function 
as well as hashing all the visited states for all of the algorithms.
"""